In [ ]:
# ==========================================================
# EXTRACCIÓN DE CARACTERÍSTICAS PARA PREDICCIÓN DE HIPOXIA
# ¿Qué se hace en extracción de características?
#Variables originales:
#oxigeno_disuelto_mg_l, temperatura_c, ph, amoniaco_mg_l, nitrato_ppm, turbidez_ntu, manganeso_mg_l.
##################Nuevas variables derivadas (ejemplos):
#Promedio, mínimo, máximo y desviación en 1 hora.
#Pendiente (tasa de cambio).
#Diferencia simple con el dato anterior.
#Suavizado (EWMA) y residuo (cuánto se desvía DO de su tendencia).
#Interacciones fisicoquímicas: temp × DO, ph × amoníaco.
#Hora del día en seno/coseno (hipoxia ocurre más de madrugada).

In [2]:
import pandas as pd
import numpy as np

In [3]:
# --- Asegurar índice de tiempo ---
df["marca_tiempo"] = pd.to_datetime(df["marca_tiempo"], errors="coerce")
df = df.sort_values("marca_tiempo").set_index("marca_tiempo")

NameError: name 'df' is not defined

In [ ]:
# --- Variables base ---
vars_base = ["oxigeno_disuelto_mg_l","temperatura_c","ph",
             "amoniaco_mg_l","nitrato_ppm","turbidez_ntu","manganeso_mg_l"]

In [ ]:
# --- Ventanas móviles (ej. 1 hora) ---
for v in vars_base:
    r = df[v].rolling("60min", min_periods=3)
    df[f"{v}_media_1h"] = r.mean()
    df[f"{v}_min_1h"]   = r.min()
    df[f"{v}_max_1h"]   = r.max()
    df[f"{v}_std_1h"]   = r.std()


In [ ]:
# --- Dinámica temporal ---
dt_h = df.index.to_series().diff().dt.total_seconds()/3600
for v in vars_base:
    df[f"{v}_pendiente_h"] = df[v].diff() / dt_h
    df[f"{v}_diferencia"]  = df[v].diff()

In [ ]:
# --- Suavizado y residuo para DO ---
df["do_suavizado"] = df["oxigeno_disuelto_mg_l"].ewm(span=12, adjust=False).mean()
df["do_residuo"]   = df["oxigeno_disuelto_mg_l"] - df["do_suavizado"]


In [ ]:
# --- Interacciones ---
df["temp_por_do"]   = df["temperatura_c"] * df["oxigeno_disuelto_mg_l"]
df["ph_por_amonio"] = df["ph"] * df["amoniaco_mg_l"]


In [ ]:
# --- Hora del día ---
df["hora"] = df.index.hour
df["hora_sin"] = np.sin(2*np.pi*df["hora"]/24)
df["hora_cos"] = np.cos(2*np.pi*df["hora"]/24)


In [1]:
# --- Limpiar NaN ---
df = df.fillna(method="bfill").fillna(method="ffill")

print("Extracción lista. Shape final:", df.shape)
print("Ejemplo de columnas:", list(df.columns)[:12])

NameError: name 'df' is not defined